In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import csv
import os
import spacy

datas = []


In [9]:
import requests

api_key = 'd7ccf52b1d89825d0a4eea78f785a707'
base_url = 'https://api.themoviedb.org/3'
image_base_url = 'https://image.tmdb.org/t/p/w500'

endpoint = '/tv/popular'

params = {
    'api_key': api_key,
    'language': 'en-US',
}
genre_endpoint = '/genre/tv/list'
genre_params = {
    'api_key': api_key,
    'language': 'en-US',
}

genre_response = requests.get(base_url + genre_endpoint, params=genre_params)

if genre_response.status_code == 200:
    nlp = spacy.load('en_core_web_sm')
    genre_data = genre_response.json()
    genre_mapping = {genre['id']: genre['name'] for genre in genre_data['genres']}
    for page in range(501,502):
        print(page)
        params['page'] = page
        response = requests.get(base_url + endpoint, params=params)
        if response.status_code == 200:
            
            data = response.json()
            for tv_show in data['results']:
                if tv_show['poster_path'] is None:
                    tv_show['poster_path'] = '/7dFZJ2ZJJdcmkp05B9NWlqTJ5tq.jpg'
                details = {
                    'name': tv_show['name'],
                    'overview': tv_show['overview'],
                    'id': tv_show['id'],
                    'poster_path': image_base_url + tv_show['poster_path'],
                    'rating':tv_show['vote_average'],
                    'vote_count':tv_show['vote_count']
                }
                tv_id = tv_show['id']
                details_endpoint = f'/tv/{tv_id}'
                details_response = requests.get(base_url + details_endpoint, params=params)
                if details_response.status_code == 200:
                    # Parse the JSON response for TV show details
                    tv_details = details_response.json()
                    
                    # Extract additional details
                    details['adult'] = tv_details['adult'] 
                    details['director'] = [crew['name'] for crew in tv_details.get('credits', {}).get('crew', []) if crew['job'] == 'Director']
                    details['producer'] = [crew['name'] for crew in tv_details.get('credits', {}).get('crew', []) if crew['job'] == 'Producer']
                    details['creator'] = [creator['name'] for creator in tv_details.get('created_by', [])]
                    details['actors'] = [cast['name'] for cast in tv_details.get('credits', {}).get('cast', [])][:5]  # Limit to top 5 actors
                    details['year_released'] = tv_details.get('first_air_date', '').split('-')[0]
                   
                genre_ids = tv_show.get('genre_ids', [])
                details['genres'] = [genre_mapping.get(genre_id, 'Unknown') for genre_id in genre_ids]
                doc = nlp(details['overview'])
                keywords = [token.text for token in doc if not token.is_stop and token.is_alpha]
                details['keywords'] = keywords

                datas.append(details)
        else:
            print(f"Failed to retrieve TV show data for page {page}. Status code:", response.status_code)
            break 
else:
    print("Failed to retrieve genre data. Status code:", genre_response.status_code)


501
Failed to retrieve TV show data for page 501. Status code: 400


In [5]:
df = pd.DataFrame(datas)
df.to_csv('id_name.csv', index=False)

In [21]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("temp.csv")

# Remove duplicate rows
df = df.drop_duplicates()

# Save the DataFrame back to a new CSV file without duplicate rows
df.to_csv("data.csv", index=False)


In [1]:
import requests

api_key = 'd7ccf52b1d89825d0a4eea78f785a707'
base_url = 'https://api.themoviedb.org/3'
image_base_url = 'https://image.tmdb.org/t/p/w500'

endpoint = '/tv/popular'

params = {
    'api_key': api_key,
    'language': 'en-US',
}
genre_endpoint = '/genre/tv/list'
genre_params = {
    'api_key': api_key,
    'language': 'en-US',
}

genre_response = requests.get(base_url + genre_endpoint, params=genre_params)

if genre_response.status_code == 200:
    genre_data = genre_response.json()
    genre_mapping = {genre['id']: genre['name'] for genre in genre_data['genres']}
    for page in range(501,1000):
        print(page)
        params['page'] = page
        response = requests.get(base_url + endpoint, params=params)
        if response.status_code == 200:  
            data = response.json()
            for tv_show in data['results']:
                if tv_show['poster_path'] is None:
                    tv_show['poster_path'] = '/7dFZJ2ZJJdcmkp05B9NWlqTJ5tq.jpg'
                details = {
                    'name': tv_show['name'],
                    'movieId': tv_show['id'],
                    'poster_path': image_base_url + tv_show['poster_path']
                }
                datas.append(details)
        else:
            print(f"Failed to retrieve TV show data for page {page}. Status code:", response.status_code)
            break 
else:
    print("Failed to retrieve genre data. Status code:", genre_response.status_code)


501
Failed to retrieve TV show data for page 501. Status code: 400


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create a Spark session
spark = SparkSession.builder.appName("ratings_matrix").getOrCreate()

# Given data
data = spark.createDataFrame([
    (1, 101, 5), (1, 102, 4),
    (2, 101, 4),
    (3, 101, 2), (3, 102, 3), (3, 103, 4),
    (4, 101, 5),
    (5, 104, 3), (5, 105, 4),
    (6, 101, 4), (6, 103, 4),
    (7, 102, 4),
    (8, 101, 4), (8, 102, 3), (8, 103, 4), (8, 104, 2), (8, 105, 3),
    (9, 101, 2), (9, 102, 3), (9, 103, 4), (9, 104, 5), (9, 105, 4),
    (10, 101, 5), (10, 102, 2), (10, 103, 1), (10, 104, 4), (10, 105, 5)
], ["userId", "movieId", "rating"])

# Pivot the DataFrame to create a matrix
ratings_matrix = data.groupBy("userId").pivot("movieId").agg({"rating": "first"})

# Fill null values with 0 (assuming unrated movies)
ratings_matrix = ratings_matrix.na.fill(0)

# Show the resulting matrix
ratings_matrix.show()


+------+---+---+---+---+---+
|userId|101|102|103|104|105|
+------+---+---+---+---+---+
|     7|  0|  4|  0|  0|  0|
|     6|  4|  0|  4|  0|  0|
|     9|  2|  3|  4|  5|  4|
|     5|  0|  0|  0|  3|  4|
|     1|  5|  4|  0|  0|  0|
|    10|  5|  2|  1|  4|  5|
|     3|  2|  3|  4|  0|  0|
|     8|  4|  3|  4|  2|  3|
|     2|  4|  0|  0|  0|  0|
|     4|  5|  0|  0|  0|  0|
+------+---+---+---+---+---+

